# web scraping RIP.ie with beautifulsoup

In [553]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datefinder

def get_dn_page(n):
    """Get death notice text from page matching the id number"""
    
    url = 'https://rip.ie/showdn.php?dn=%s' %n
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    title  = soup.title.text.strip()
    name=''    
    for s in ['Death Notice of ','The death has occurred of ']:
        if title.startswith(s):
            name = title.split(s)[1]   
    elem = soup.find_all("div", id="dn_photo_and_text")
    
    if len(elem) == 0:
        return name, '', '', '', ''
    rows = elem[0].find_all('p')
    if len(rows) == 0:
        rows = elem[0].find_all('td')
    text = ';'.join([r.text.strip() for r in rows]).replace('\n','')
    #address
    addrelem = soup.find("span", class_='small_addr') 
    if addrelem != None:
        address = addrelem.text.strip()
    else:
        address = ''
    #county  
    ctyelem = soup.find("li", class_='fd_county') 
    if ctyelem != None:
        county = ctyelem.text.strip()
    else:
        county = ''
    #date
    dateelem = soup.find("div", class_='ddeath')
    if dateelem == None:
        dateelem = soup.find("div", class_='dpubl')
    s = dateelem.text.strip()
    try:
        date = list(datefinder.find_dates(s))[0]
    except:
        date = ''
    print (n, date, name, address, county)
    return name, date, county, address, text

In [ ]:
get_dn_page(390045)

## iterate over a range of ids and get info

In [ ]:
#read current table in so we skip those already done
df = pd.read_csv('rip_dn_scrape.csv',index_col=0)
ids = list(df.index)

results={}
for n in range(308410,461389):
    if n in ids:
        continue
    name,date,cty,addr,txt = get_dn_page(n)
    if text != '':
        results[n] = [name,date,cty,addr,txt]
    time.sleep(0.05)

In [ ]:
res = pd.DataFrame.from_dict(results,orient='index',columns=['name','date','county','address','notice'])

In [606]:
res

,name,date,address,notice,county
390000,John Rushe Snr,2019-06-04,"Pearse Ave., Mervue, Galway / Galway City, Ga...","Formerly Grattan Tce., The Claddagh. Beloved h...",NaN
390001,Sheila Higgins (née Kenny),2019-06-05,"Lombard Street West, Dublin 8, Dublin","Higgins (née Kenny), Sheila, 5th June 2019, O...",NaN
390002,Fintan (Millhouse) Maguire,2019-06-04,"Kilmessan, Meath","Maguire (Millhouse) Fintan, Kilmessan, Co. Mea...",NaN
390003,Robert Scott,2019-06-05,"Blackrock, Cork / Wexford","SCOTT (Cork, Wexford and Leeds): On June 5, 2...",NaN
390004,Nora Ryan (née Griffin),2019-06-05,"Ballykillaboy, Kilmacow, Kilkenny",Wife of the late Johnny. Deeply regretted by h...,NaN
...,...,...,...,...,...
348575,Pauline O'Brien,2018-03-06 00:00:00,"Mullingar Rd., Tyrrellspass, Westmeath","Pauline O'Brien, 6th March 2018 died peacefull...",Westmeath
348576,Ann Alldritt (née Boylan),2018-03-06 00:00:00,"Devere Court, Carew Park, Limerick City, Lime...","Ann Alldritt nee Boylan, Devere Court, Carew P...",Limerick
348577,Darren Fitzpatrick,2018-03-06 00:00:00,"Gillardstown, Collinstown, Westmeath / Kildar...","Darren Fitzpatrick, Gillardstown, Collinstown,...",Westmeath
348578,Bill REID,2018-03-05 00:00:00,"Clogher Road, Dublin 12, Dublin","REID, Bill (Lanzarote and formerly of Clogher ...",Dublin


In [603]:
res = pd.concat([df,res])
res.to_csv('rip_dn_scrape.csv')

In [604]:
x=res.replace('',None).dropna(how='any')
x['date'] = pd.to_datetime(x['date']).apply(lambda x: x.strftime('%d/%m/%Y'))
x=x.drop_duplicates(['name','notice'])
print (len(res),len(x))
x.to_csv('rip_dn_scrape_processed.csv')

95711 73311


## test matching

In [382]:
targets=pd.DataFrame([('Lottie Graham','BigPark, Ballintra, Co Donegal'),('a','b')],columns=('name','address'))

In [ ]:
targets

def find_match(x,s):
    if s in str(x.notice):
        return True
    return False

s='Rialto'
x=res.apply(lambda x: find_match(x,s),1)
res[x]